In [ ]:
import pandas as pd
import numpy as np
PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/ai_optimizer/data/'

In [ ]:
df = pd.read_csv(PATH+'targeting_list_facebook.csv',encoding='gbk')

In [ ]:
df['Level 1'].unique()

In [ ]:
import os
os.getcwd()
level_1 = None
level_2 = None
df = df.replace(np.nan, df.replace([np.nan], [None]))
df['Level 3'] = df['Level 3'].str.split("?",n=1, expand=True)[0]
df_temp = pd.DataFrame(columns=df.columns)
for i, row in df.iterrows():
    if row[0] is not None and row[1] is not None:
        level_1 = row[0]
        level_2 = row[1]
    elif row[0] is None and row[1] is not None:
        row[0] = level_1
        level_2 = row[1]
    elif row[0] is not None and row[1] is None:
        row[1] = level_2
    else:
        row[0] = level_1
        row[1] = level_2
    df_temp = df_temp.append(row)


In [ ]:
df_temp.to_csv(PATH+'targeting_interests.csv', index=False, encoding='utf-8')

In [ ]:
len(df_temp['Level 2'][df_temp['Level 1']=='Business and Industry'].unique())

In [ ]:
import json

In [ ]:
level_1_list = df_temp['Level 1'].unique()
targeting_interests = list()
for level_1 in level_1_list:
    level_2_list = df_temp['Level 2'][df_temp['Level 1'] == level_1].unique()
#     print(level_2_list)
    targeting_interests.append( {'level 1': level_1, 'value': []} )
    interests = {'level 1': level_1, 'value': []}
    for level_2 in level_2_list:
        level_3_list = list( df_temp['Level 3'][(df_temp['Level 1'] == level_1)&(df_temp['Level 2'] == level_2)].unique() )
        if level_1 == interests['level 1']:
            interests['value'].append( { 'level 2': level_2, 'level 3': level_3_list} )
    interests = json.dumps(interests)
    print(interests)
    print('-'*30)   

In [3]:
def search_target_id(keyword=None):
    from facebook_business.adobjects.targetingsearch import TargetingSearch
    params = {
        'q': str(keyword),
        'type': TargetingSearch.TargetingSearchTypes.interest,
    }
    resp = TargetingSearch.search(params=params)
    current_resp = resp[0]
    interest_pair = {
        "interests":[{
            "id": current_resp["id"],
            "name": current_resp["name"],
        }]
    }
    return interest_pair

search_target_id('Lingerie')
# test_param = retrieve_origin_adset_params(23843986717420337)
# # for level_1 in level_1_list:
# #     print(search_target_id(keyword=level_1))
# interest_pair = search_target_id(keyword="Business and industry")
# interest_pair

# origin_name = test_param[AdSet.Field.name]
# test_param[AdSet.Field.name] = origin_name + "_Business and industry"
# test_param[AdSet.Field.targeting]["flexible_spec"] = interest_pair
# test_param[AdSet.Field.id] = None
# test_param

FacebookBadObjectError: An Api instance must be provided as an argument or set as the default Api in FacebookAdsApi.